# Initial Data: Fishbone-Moncrief

This goal of this module will be to construct Fishbone-Moncrief initial data for GRMHD simulations in a format suitable for the Einstein Toolkit (ETK). We will be using the equations as derived in [the original paper](http://articles.adsabs.harvard.edu/cgi-bin/nph-iarticle_query?1976ApJ...207..962F&amp;data_type=PDF_HIGH&amp;whole_paper=YES&amp;type=PRINTER&amp;filetype=.pdf), which will hereafter be called "the FM paper". To that end, we will use Cartesian coordinates, and thus not need to take advantage of the reference metric tools that NRPy+ provides.

We will begin, as usual, by importing the packages and NRPy+ modules that we will need. We will also set some of the most commonly used parameters.

In [1]:
# Step 1a: Import needed NRPy+ core modules:
import NRPy_param_funcs as par
import indexedexp as ixp
import grid as gri
import finite_difference as fin
from outputC import *
import loop

#Set the spatial dimension parameter to 3.
par.set_parval_from_str("grid::DIM", 3)
DIM = par.parval_from_str("grid::DIM")


With NRPy's most important functions now available to us, we can start to set up the rest of the tools we will need to build the initial data. We will now register the gridfunctions we expect to use. Critically, we register the physical metric and extrinsic curvature tensors. We will also invert the physical metric and take the determinant.

In [2]:
gammaDD = ixp.register_gridfunctions_for_single_rank2("AUX","gammaDD", "sym01") 
kDD = ixp.register_gridfunctions_for_single_rank2("AUX","kDD", "sym01")
gammaUU, detgamma = ixp.symm_matrix_inverter3x3(gammaDD)

r_in,r_at_max_density,a,M,l = gri.register_gridfunctions("EVOL",["r_in","r_at_max_density","a","M","l"])
kappa,gamma = gri.register_gridfunctions("EVOL",["kappa","gamma"])

LorentzFactor = gri.register_gridfunctions("AUX","LorentzFactor")

Now, we can begin actually building the ID equations. We will start with the value of the angular momentum $l$ at the position $r \equiv \text{r_at_max_density}$ where the density is at a maximum, as in equation 3.8 of the FM paper:
\begin{align}
l(r) &= \pm \left( \frac{M}{r^3} \right) ^{1/2}
    \left[ \frac{r^4+r^2a^2-2Mra^2 \mp a(Mr)^{1/2}(r^2-a^2)}
    {r^2 -3Mr \pm 2a(Mr)^{1/2}} \right].
\end{align}

In [4]:
# First compute angular momentum at r_at_max_density, TAKING POSITIVE ROOT. This way disk is co-rotating with black hole
# Eq 3.8:
l_r  = sp.sqrt(M/r_at_max_density**3) * (r_at_max_density**4 + r_at_max_density**2*a**2 - 2*M*r_at_max_density*a**2 - \
                                         a*sp.sqrt(M*r_at_max_density)*(r_at_max_density**2-a**2))
l_r /= r_at_max_density**2 - 3*M*r_at_max_density + 2*a*sp.sqrt(M*r_at_max_density)

l = l_r

Next, we will follow equation 3.6 of the FM paper to compute the enthalpy $h$ by first finding its logarithm $\ln h$. Fortunately, we can make this process quite a bit simpler by first identifying the common subexpressions.

In [ ]:
    # Eq 3.6:
    # First compute the radially-independent part of the log of the enthalpy, ln_h_const
    Delta = r**2 - 2*M*r + a**2
    Sigma = r**2 + a**2*cos(th)**2
    A = (r**2 + a**2)**2 - Delta*a**2*sin(th)**2

    # Next compute the radially-dependent part of log(enthalpy), ln_h
    tmp3 = sqrt(1 + 4*l**2*Sigma**2*Delta/(A*sin(th))**2)
    # Term 1 of Eq 3.6
    ln_h  = Rational(1,2)*log( ( 1 + tmp3) / (Sigma*Delta/A))
    # Term 2 of Eq 3.6
    ln_h -= Rational(1,2)*tmp3
    # Term 3 of Eq 3.6
    ln_h -= 2*a*M*r*l/A


    # Next compute the radially-INdependent part of log(enthalpy), ln_h
    # Note that there is some typo in the expression for these terms given in Eq 3.6, so we opt to just evaluate
    #   negative of the first three terms at r=r_in and th=pi/2 (the integration constant), as described in
    #   the text below Eq. 3.6, basically just copying the above lines of code.
    # Delin = Delta_in ; Sigin = Sigma_in ; Ain = A_in .
    Delin = r_in**2 - 2*M*r_in + a**2
    Sigin = r_in**2 + a**2*cos(pi/2)**2
    Ain   = (r_in**2 + a**2)**2 - Delin*a**2*sin(pi/2)**2

    tmp3in = sqrt(1 + 4*l**2*Sigin**2*Delin/(Ain*sin(pi/2))**2)
    # Term 4 of Eq 3.6
    mln_h_in  = -Rational(1,2)*log( ( 1 + tmp3in) / (Sigin*Delin/Ain))
    # Term 5 of Eq 3.6
    mln_h_in += Rational(1,2)*tmp3in
    # Term 6 of Eq 3.6
    mln_h_in += 2*a*M*r_in*l/Ain

    hm1 = exp(ln_h + mln_h_in) - 1
